In [316]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [319]:
import sys;
sys.path.append(r'C:\Monitor\\')
sys.path.append(r'\\ac-geneva-24\E\grains trading\Streamlit\Monitor\\')

from datetime import datetime as dt
from copy import deepcopy

import numpy as np
import pandas as pd; pd.options.mode.chained_assignment = None
import Utilities.Modeling as um
import Utilities.Weather as uw
import Utilities.GLOBAL as GV
import Utilities.Modeling as um
import Models.Corn_USA_Yield as cy
import Utilities.SnD as us

import plotly.express as px

In [328]:
prec_units='mm'
temp_units='C'

scope = cy.Define_Scope()
# raw_data = cy.Get_Data_All_Parallel(scope)
# raw_data.keys()

w_df_all=uw.build_w_df_all(scope['geo_df'], scope['w_vars'], scope['geo_input_file'], scope['geo_output_column'])
weights=us.get_USA_prod_weights('CORN', 'STATE', scope['years'], scope['geo_df'][GV.WS_STATE_ALPHA])

uw.weighted_w_df_all(w_df_all, weights, output_column='USA')


{'hist':             USA_Sdd30  USA_TempMax   USA_Prec
 time                                         
 1985-01-01   0.000000    -1.383573  11.863999
 1985-01-02   0.000000    -3.520978   0.568994
 1985-01-03   0.000000    -4.898498   0.047351
 1985-01-04   0.000000    -1.346037   0.973402
 1985-01-05   0.000000     2.471312   0.224217
 ...               ...          ...        ...
 2022-06-24   1.664973    31.575519   1.084581
 2022-06-25   0.996314    30.731462   8.365321
 2022-06-26   0.282337    28.017786   6.009647
 2022-06-27   0.000000    25.487994   1.139391
 2022-06-28   0.000000    27.379394   0.202332
 
 [13693 rows x 3 columns],
 'gfs':             USA_Sdd30  USA_TempMax  USA_Prec
 time                                        
 2022-06-29   1.637770    30.070637  0.049771
 2022-06-30   0.824638    29.877877  0.130165
 2022-07-01   0.496204    28.607603  3.364279
 2022-07-02   0.000000    25.821741  1.935233
 2022-07-03   0.102118    26.758895  0.097654
 2022-07-04   1.164850 

In [ ]:
milestones =cy.Milestone_from_Progress(raw_data)
intervals = cy.Intervals_from_Milestones(milestones)

train_DF_instr = um.Build_DF_Instructions('weighted',GV.WD_HIST, prec_units=prec_units, temp_units=temp_units, ext_mode = GV.EXT_DICT)
train_df = cy.Build_DF(raw_data, milestones, intervals, train_DF_instr)

model = um.Fit_Model(train_df,'Yield',GV.CUR_YEAR)
# print(model.summary())

In [ ]:
train_df.columns

In [ ]:
model.params

In [ ]:
step=2.0

col=['Planting Prec']
min_x = train_df[col].min()
max_x = train_df[col].max()

pp_x = np.arange(0,1.0*max_x.values,step)
pp_y = model.params['Planting Prec']*pp_x+ model.params['Planting Prec Squared']*pp_x*pp_x
fig=px.line(x=pp_x, y=pp_y)
# fig.show('browser')
print(pp_x.max())
print(pp_x.shape)


col=['Jul Aug Prec']
min_x = train_df[col].min()
max_x = train_df[col].max()

jp_x = np.arange(0,1.0*max_x.values,step)
jp_y = model.params['Jul Aug Prec']*jp_x+ model.params['Jul Aug Prec Squared']*jp_x*jp_x
fig=px.line(x=jp_x, y=jp_y)
# fig.show('browser')
print(jp_x.max())
print(jp_x.shape)


interaction = np.matrix(pp_x).T * np.matrix(jp_x) * model.params['Prec Interaction']
prec = np.matrix(pp_y).T + np.matrix(jp_y)
all_prec = prec+interaction


fig=px.imshow(all_prec,x=jp_x,y=pp_x)
fig.update_yaxes(autorange=True) 
# fig=px.imshow(all_prec)
fig.show('browser')
print(np.max(all_prec))

In [331]:
just_weather=uw.build_w_df_all(scope['geo_df'], scope['w_vars'], scope['geo_input_file'], scope['geo_output_column'])
# weighted= uw.weighted_w_df_all(just_weather, raw_data['weights'], output_column='USA')
just_weather.keys()

dict_keys(['hist', 'gfs', 'ecmwf', 'hist_gfs', 'hist_ecmwf'])

In [332]:
just_weather['gfs'].to_csv('dm_GFS.csv')